In [2]:
from flask import Flask, request, render_template
import tensorflow as tf
import numpy as np
from PIL import Image
import os
from werkzeug.utils import secure_filename


model = tf.keras.models.load_model("1.keras")

# Define the class names (update based on your training data classes)
class_names = ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

# Initialize the Flask app
app = Flask(__name__)

# Preprocess the uploaded image
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = img.resize((224, 224))  # Resize to model's input size
    img_array = np.array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Prediction function
def predict(image_path):
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * np.max(predictions[0]), 2)
    return predicted_class, confidence

# Routes
@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        # Check if a file is uploaded
        if "file" not in request.files or request.files["file"].filename == "":
            return "No file uploaded", 400

        file = request.files["file"]
        filename = secure_filename(file.filename)

        # Save the uploaded file
        file_path = os.path.join("static", filename)
        os.makedirs("static", exist_ok=True)  # Ensure the 'static' folder exists
        file.save(file_path)

        # Make a prediction
        predicted_class, confidence = predict(file_path)

        # Render result page
        return render_template(
            "result.html",
            image_path=file_path,
            prediction=predicted_class,
            confidence=confidence,
        )

    return render_template("index.html")

# Run Flask app from Jupyter
import threading

def run_flask():
    app.run(debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Nov/2024 15:22:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:22:11] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step


127.0.0.1 - - [28/Nov/2024 15:22:22] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:22:22] "GET /static/0a8a68ee-f587-4dea-beec-79d02e7d3fa4___RS_Early.B_8461.JPG HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 15:22:22] "GET / HTTP/1.1" 200 -


In [23]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the trained model
model = tf.keras.models.load_model("2.keras")  # Ensure the model file name matches
print("Model loaded successfully.")

# Define the class names as per the training data
class_names = ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

# Define the image preprocessing function (same as in the app)
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = img.resize((224, 224))  # Match model's expected input size
    img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Define the predict function (same as in the app)
def predict(image_path):
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)  # Get predictions from the model
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * np.max(predictions[0]), 2)
    return predicted_class, confidence

# Test the predict function with a sample image
test_image_path = "C:/Users/03abd/poatato desesasis/00fc2ee5-729f-4757-8aeb-65c3355874f2___RS_HL 1864.JPG"  # Replace with the path to a test image
try:
    predicted_class, confidence = predict(test_image_path)
    print(f"Prediction Result for {test_image_path}:")
    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence: {confidence}%")
except Exception as e:
    print(f"Error: {e}")

Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
Prediction Result for C:/Users/03abd/poatato desesasis/00fc2ee5-729f-4757-8aeb-65c3355874f2___RS_HL 1864.JPG:
Predicted Class: Potato___Early_blight
Confidence: 100.0%
